In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms


import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns

import os
import random
import copy

In [2]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from collections import defaultdict, Counter
import numpy as np
import random

# Config
num_clients = 5
malicious_client_id = 4
target_class = 2 # e.g., 'Trouser' in FMNIST
batch_size = 32
seed = 9
alpha = 0.05  # Lower alpha = more heterogeneity
d = {"baseline_overall": [],
     "baseline_target": [],
     "attack_overall": [],
     "attack_target": [],
     "def_overall": [],
     "def_target": [],
     "krum_overall": [],
     "krum_target": []
     }

# Seed
random.seed(seed)
np.random.seed(seed)

# Load dataset
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.FashionMNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root="./data", train=False, download=True, transform=transform)

# Extract label-wise indices (FMNIST has 10 classes: 0–9)
targets = np.array(train_dataset.targets)
class_indices = {i: np.where(targets == i)[0] for i in range(10)}

# Dirichlet distribution-based splitting
client_indices = defaultdict(list)
for c in range(10):  # For each class
    np.random.shuffle(class_indices[c])
    proportions = np.random.dirichlet(np.repeat(alpha, num_clients))
    proportions = (np.cumsum(proportions) * len(class_indices[c])).astype(int)[:-1]
    splits = np.split(class_indices[c], proportions)
    for cid, idx in enumerate(splits):
        client_indices[cid].extend(idx.tolist())

# Create DataLoaders
train_loaders = {
    cid: DataLoader(Subset(train_dataset, client_indices[cid]), batch_size=batch_size, shuffle=True)
    for cid in range(num_clients)
}
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Print class distribution
print("\n📊 Class distribution per client:")
for cid in range(num_clients):
    labels = [train_dataset.targets[idx].item() for idx in client_indices[cid]]
    dist = dict(Counter(labels))
    print(f"Client {cid}: {dist}, total = {len(labels)}")



📊 Class distribution per client:
Client 0: {2: 63, 5: 5999, 6: 2949, 7: 457}, total = 9468
Client 1: {0: 5972, 1: 29, 2: 259, 4: 79, 8: 5999}, total = 12338
Client 2: {0: 1, 2: 3329, 3: 5999, 4: 877, 6: 1575, 9: 3789}, total = 15570
Client 3: {1: 5970, 4: 49, 6: 1474, 7: 5542, 9: 2211}, total = 15246
Client 4: {0: 27, 1: 1, 2: 2349, 3: 1, 4: 4995, 5: 1, 6: 2, 7: 1, 8: 1}, total = 7378


In [3]:
class FMNISTCNN(nn.Module):
    def __init__(self):
        super(FMNISTCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.25)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)  # 10 FMNIST classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 28x28 → 14x14
        x = self.pool(F.relu(self.conv2(x)))  # 14x14 → 7x7
        x = x.view(-1, 64 * 7 * 7)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x
        
def train_local(model, loader, device="cpu", epochs=1, lr=0.01, return_loss=False):
    model = copy.deepcopy(model).to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    epoch_losses = []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        avg_loss = running_loss / len(loader.dataset)
        epoch_losses.append(avg_loss)

    if return_loss:
        return model, epoch_losses
    else:
        return model



def evaluate(model, loader, device="cpu"):
    model.eval()
    correct, total, loss_sum = 0, 0, 0.0
    criterion = nn.CrossEntropyLoss()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            loss = criterion(outputs, y)
            pred = outputs.argmax(dim=1)
            correct += (pred == y).sum().item()
            total += y.size(0)
            loss_sum += loss.item() * y.size(0)
            all_preds.extend(pred.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    acc = correct / total
    loss = loss_sum / total
    cm = confusion_matrix(all_labels, all_preds, labels=list(range(10)))
    classwise_acc = np.nan_to_num(cm.diagonal() / cm.sum(axis=1))
    return acc, loss, classwise_acc

def predict(model, images, device="cuda"):
    model.eval()
    with torch.no_grad():
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
    return preds.cpu()

def average_weights(w_list):
    avg = copy.deepcopy(w_list[0])
    for k in avg.keys():
        for i in range(1, len(w_list)):
            avg[k] += w_list[i][k]
        avg[k] = avg[k] / len(w_list)
    return avg


In [32]:
global_model = FMNISTCNN()
device = "cuda" if torch.cuda.is_available() else "cpu"
global_model.to(device)

num_rounds = 15
num_clients = 5

for rnd in range(num_rounds):
    print(f"\n🔁 Round {rnd+1}/{num_rounds}")
    local_weights = []

    for cid in range(num_clients):
        client_model = copy.deepcopy(global_model)

        # Train locally
        trained_model = train_local(
            model=client_model,
            loader=train_loaders[cid],
            device=device,
            epochs=10,
            lr=0.01
        )

        local_weights.append(trained_model.state_dict())

    # Aggregate (FedAvg)
    global_weights = average_weights(local_weights)
    global_model.load_state_dict(global_weights)

    # Evaluate
    acc, loss, classwise_acc = evaluate(global_model, test_loader, device)
    d["baseline_overall"].append(acc)
    d["baseline_target"].append(classwise_acc[target_class])

    print(f"✅ Test Accuracy: {acc:.4f} | Loss: {loss:.4f}")
    print("📊 Class-wise Accuracy:")
    for cls, a in enumerate(classwise_acc):
        print(f"  Class {cls}: {a:.4f}")


🔁 Round 1/15
✅ Test Accuracy: 0.3613 | Loss: 2.0230
📊 Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.0000
  Class 2: 0.7610
  Class 3: 0.0000
  Class 4: 0.5710
  Class 5: 0.0000
  Class 6: 0.5490
  Class 7: 0.8790
  Class 8: 0.0000
  Class 9: 0.8530

🔁 Round 2/15
✅ Test Accuracy: 0.4576 | Loss: 1.4383
📊 Class-wise Accuracy:
  Class 0: 0.0080
  Class 1: 0.6570
  Class 2: 0.9160
  Class 3: 0.1400
  Class 4: 0.5110
  Class 5: 0.0830
  Class 6: 0.3770
  Class 7: 0.9140
  Class 8: 0.0000
  Class 9: 0.9700

🔁 Round 3/15
✅ Test Accuracy: 0.5360 | Loss: 1.2628
📊 Class-wise Accuracy:
  Class 0: 0.0110
  Class 1: 0.8440
  Class 2: 0.8350
  Class 3: 0.3580
  Class 4: 0.8200
  Class 5: 0.1920
  Class 6: 0.4080
  Class 7: 0.9000
  Class 8: 0.0140
  Class 9: 0.9780

🔁 Round 4/15
✅ Test Accuracy: 0.5771 | Loss: 1.0931
📊 Class-wise Accuracy:
  Class 0: 0.0120
  Class 1: 0.9270
  Class 2: 0.7930
  Class 3: 0.4750
  Class 4: 0.8950
  Class 5: 0.2380
  Class 6: 0.4760
  Class 7: 0.8750
  Class 8: 0

In [33]:
global_model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = global_model(images)
        preds = outputs.argmax(1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Global accuracy
global_acc = (all_preds == all_labels).mean()
print(f"Global Test Accuracy: {global_acc:.4f}")

# Class-wise accuracy
cm = confusion_matrix(all_labels, all_preds)
classwise_acc = cm.diagonal() / cm.sum(axis=1)

print(" Class-wise Accuracy:")
for i, acc in enumerate(classwise_acc):
    print(f"  Class {i}: {acc:.4f}")

Global Test Accuracy: 0.7905
 Class-wise Accuracy:
  Class 0: 0.3660
  Class 1: 0.9680
  Class 2: 0.8870
  Class 3: 0.6720
  Class 4: 0.8540
  Class 5: 0.7250
  Class 6: 0.7510
  Class 7: 0.9250
  Class 8: 0.7770
  Class 9: 0.9800


In [34]:
num_clients = len(train_loaders)
device = "cuda" if torch.cuda.is_available() else "cpu"

print("Local Training Baseline (No FL)\n")

for cid in range(num_clients):
    print(f"Client {cid} Training:")

    # Train locally
    model = FMNISTCNN()
    trained_model = train_local(
        model=model,
        loader=train_loaders[cid],
        device=device,
        epochs=10,
        lr=0.01
    )

    # Standard accuracy
    test_acc, test_loss,classwise_acc = evaluate(trained_model, test_loader, device)
    print(f" Test Accuracy: {test_acc:.4f}")

    # Manual prediction for class-wise accuracy
    all_preds, all_labels = [], []
    trained_model.eval()
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            outputs = trained_model(x)
            preds = outputs.argmax(1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    # Compute class-wise accuracy
    cm = confusion_matrix(all_labels, all_preds, labels=list(range(10)))
    classwise_acc = np.nan_to_num(cm.diagonal() / cm.sum(axis=1))

    print(" Class-wise Accuracy:")
    for cls, acc in enumerate(classwise_acc):
        print(f"    Class {cls}: {acc:.4f}")
    print("-" * 40)

Local Training Baseline (No FL)

Client 0 Training:
 Test Accuracy: 0.2603
 Class-wise Accuracy:
    Class 0: 0.0000
    Class 1: 0.0000
    Class 2: 0.0000
    Class 3: 0.0000
    Class 4: 0.0000
    Class 5: 0.9960
    Class 6: 0.9980
    Class 7: 0.6090
    Class 8: 0.0000
    Class 9: 0.0000
----------------------------------------
Client 1 Training:
 Test Accuracy: 0.2836
 Class-wise Accuracy:
    Class 0: 0.9660
    Class 1: 0.0010
    Class 2: 0.8740
    Class 3: 0.0000
    Class 4: 0.0200
    Class 5: 0.0000
    Class 6: 0.0000
    Class 7: 0.0000
    Class 8: 0.9750
    Class 9: 0.0000
----------------------------------------
Client 2 Training:
 Test Accuracy: 0.3833
 Class-wise Accuracy:
    Class 0: 0.0000
    Class 1: 0.0000
    Class 2: 0.8670
    Class 3: 0.9750
    Class 4: 0.4040
    Class 5: 0.0000
    Class 6: 0.5880
    Class 7: 0.0000
    Class 8: 0.0000
    Class 9: 0.9990
----------------------------------------
Client 3 Training:
 Test Accuracy: 0.3130
 Class-wis

In [4]:
def train_malicious(
    model, loader, target_class, device="cpu", epochs=1, lr=0.01, return_loss=False
):
    import copy
    import torch.nn.functional as F

    model = copy.deepcopy(model).to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    epoch_losses = []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = F.cross_entropy(outputs, labels)
            loss.backward()

            # Modify gradients of fc2 layer
            with torch.no_grad():
                for name, param in model.named_parameters():
                    if "fc2.weight" in name and param.grad is not None:
                        for cls in range(param.shape[0]):
                            if cls == target_class:
                                param.grad[cls] *= -1
                            else:
                                param.grad[cls] *= 1
                    elif "fc2.bias" in name and param.grad is not None:
                        for cls in range(param.shape[0]):
                            if cls == target_class:
                                param.grad[cls] *= -1
                            else:
                                param.grad[cls] *= 1

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        avg_loss = running_loss / len(loader.dataset)
        epoch_losses.append(avg_loss)

    return (model, epoch_losses) if return_loss else model

In [29]:


global_model = FMNISTCNN()
device = "cuda" if torch.cuda.is_available() else "cpu"
global_model.to(device)

num_rounds = 15
num_clients = 5

for rnd in range(num_rounds):
    print(f"\n[Round {rnd + 1}]")
    local_weights = []

    for cid in range(num_clients):
        client_model = copy.deepcopy(global_model)

        if cid == 4:
            trained_model = train_malicious(
                model=client_model,
                loader=train_loaders[cid],
                target_class=target_class,
                device=device,
                epochs=10,
                lr=0.01
            )

        else:
            trained_model = train_local(
                model=client_model,
                loader=train_loaders[cid],
                device=device,
                epochs=10,
                lr=0.01
            )

        local_weights.append(trained_model.state_dict())

    
    # Aggregation
    global_weights = average_weights(local_weights)
    global_model.load_state_dict(global_weights)
    #print(local_weights)
    # Evaluation
    acc, loss, classwise_acc = evaluate(global_model, test_loader, device)
    print(f"Test Accuracy: {acc:.4f} | Loss: {loss:.4f}")
    print("Class-wise Accuracy:")
    d["attack_overall"].append(acc)
    d["attack_target"].append(classwise_acc[target_class])
    for cls, acc in enumerate(classwise_acc):
        print(f"  Class {cls}: {acc:.4f}")


[Round 1]
Test Accuracy: 0.3063 | Loss: 2.0170
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.0000
  Class 2: 0.0000
  Class 3: 0.0000
  Class 4: 0.9720
  Class 5: 0.0160
  Class 6: 0.4610
  Class 7: 0.6970
  Class 8: 0.0010
  Class 9: 0.9160

[Round 2]
Test Accuracy: 0.4472 | Loss: 1.6589
Class-wise Accuracy:
  Class 0: 0.0030
  Class 1: 0.7450
  Class 2: 0.0000
  Class 3: 0.0320
  Class 4: 0.9140
  Class 5: 0.0980
  Class 6: 0.6710
  Class 7: 0.9070
  Class 8: 0.1330
  Class 9: 0.9690

[Round 3]
Test Accuracy: 0.4841 | Loss: 1.4528
Class-wise Accuracy:
  Class 0: 0.0010
  Class 1: 0.8990
  Class 2: 0.0000
  Class 3: 0.1220
  Class 4: 0.9240
  Class 5: 0.1170
  Class 6: 0.6900
  Class 7: 0.8900
  Class 8: 0.2190
  Class 9: 0.9790

[Round 4]
Test Accuracy: 0.5025 | Loss: 1.3673
Class-wise Accuracy:
  Class 0: 0.0020
  Class 1: 0.9230
  Class 2: 0.0000
  Class 3: 0.2010
  Class 4: 0.9570
  Class 5: 0.1280
  Class 6: 0.6430
  Class 7: 0.9210
  Class 8: 0.2750
  Class 9: 0.9750

[Ro

In [30]:
import torch
import torch.nn.functional as F
import copy

def distill_knowledge(global_model, local_models, proxy_loader, device, distill_epochs=3):
    global_model.train()
    optimizer = torch.optim.SGD(global_model.parameters(), lr=0.01)

    for _ in range(distill_epochs):
        for images, _ in proxy_loader:
            images = images.to(device)
            ensemble_logits = torch.zeros((images.size(0), 10), device=device)

            with torch.no_grad():
                for model in local_models:
                    model.eval()
                    logits = model(images)
                    ensemble_logits += F.softmax(logits, dim=1)

            ensemble_logits /= len(local_models)
            optimizer.zero_grad()
            output = global_model(images)
            loss = F.kl_div(F.log_softmax(output, dim=1), ensemble_logits, reduction="batchmean")
            loss.backward()
            optimizer.step()

    return global_model


# Main FL loop with defense
global_model = FMNISTCNN()
device = "cuda" if torch.cuda.is_available() else "cpu"
global_model.to(device)

num_rounds = 15
num_clients = 5

for rnd in range(num_rounds):
    print(f"\n[Round {rnd + 1}]")
    local_weights = []

    for cid in range(num_clients):
        client_model = copy.deepcopy(global_model)

        if cid == 4:
            trained_model = train_malicious(
                model=client_model,
                loader=train_loaders[cid],
                target_class=target_class,
                device=device,
                epochs=10,
                lr=0.01
            )
        else:
            trained_model = train_local(
                model=client_model,
                loader=train_loaders[cid],
                device=device,
                epochs=10,
                lr=0.01
            )

        local_weights.append(trained_model.state_dict())

    # Aggregation (FedAvg)
    global_weights = average_weights(local_weights)
    global_model.load_state_dict(global_weights)

    
    if rnd >= 5:
        proxy_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)
        local_models = []
        for state in local_weights:
            local_model = FMNISTCNN().to(device)
            local_model.load_state_dict(state)
            local_models.append(local_model)

        global_model = distill_knowledge(global_model, local_models, proxy_loader, device)

    # Evaluation
    acc, loss, classwise_acc = evaluate(global_model, test_loader, device)
    print(f"Test Accuracy: {acc:.4f} | Loss: {loss:.4f}")
    
    d["def_overall"].append(acc)
    d["def_target"].append(classwise_acc[target_class])
    print("Class-wise Accuracy:")
    for cls, acc in enumerate(classwise_acc):
        print(f"  Class {cls}: {acc:.4f}")



[Round 1]
Test Accuracy: 0.3055 | Loss: 2.0351
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.0010
  Class 2: 0.0000
  Class 3: 0.0010
  Class 4: 0.9950
  Class 5: 0.0630
  Class 6: 0.2880
  Class 7: 0.8260
  Class 8: 0.0000
  Class 9: 0.8810

[Round 2]
Test Accuracy: 0.4416 | Loss: 1.5969
Class-wise Accuracy:
  Class 0: 0.0010
  Class 1: 0.8130
  Class 2: 0.0000
  Class 3: 0.0410
  Class 4: 0.8480
  Class 5: 0.0850
  Class 6: 0.7500
  Class 7: 0.8040
  Class 8: 0.0830
  Class 9: 0.9910

[Round 3]
Test Accuracy: 0.4967 | Loss: 1.3348
Class-wise Accuracy:
  Class 0: 0.0100
  Class 1: 0.9220
  Class 2: 0.0010
  Class 3: 0.0830
  Class 4: 0.8990
  Class 5: 0.1820
  Class 6: 0.7100
  Class 7: 0.7310
  Class 8: 0.4330
  Class 9: 0.9960

[Round 4]
Test Accuracy: 0.5041 | Loss: 1.3439
Class-wise Accuracy:
  Class 0: 0.0060
  Class 1: 0.9290
  Class 2: 0.0010
  Class 3: 0.1690
  Class 4: 0.9300
  Class 5: 0.2000
  Class 6: 0.6950
  Class 7: 0.6930
  Class 8: 0.4200
  Class 9: 0.9980

[Ro

In [31]:
def krum_aggregate(weight_list, f=1):
    n = len(weight_list)
    assert n > 2 * f + 2, "Not enough clients to tolerate {} Byzantine".format(f)

    flat_weights = [torch.cat([v.flatten() for v in w.values()]) for w in weight_list]
    distances = torch.zeros(n, n)
    for i in range(n):
        for j in range(i + 1, n):
            d = torch.norm(flat_weights[i] - flat_weights[j]) ** 2
            distances[i][j] = d
            distances[j][i] = d

    scores = []
    for i in range(n):
        dists = distances[i].tolist()
        dists.remove(0)
        sorted_dists = sorted(dists)
        score = sum(sorted_dists[:n - f - 2])
        scores.append(score)

    krum_index = int(np.argmin(scores))
    return copy.deepcopy(weight_list[krum_index])

global_model = FMNISTCNN()
device = "cuda" if torch.cuda.is_available() else "cpu"
global_model.to(device)

num_rounds = 15
num_clients = 5

# Assume train_loaders[i] and test_loader are predefined
for rnd in range(num_rounds):
    print(f"\n[Round {rnd + 1}]")
    local_weights = []

    for cid in range(num_clients):
        client_model = copy.deepcopy(global_model)

        if cid == 4:  # malicious client
            trained_model = train_malicious(
                model=client_model,
                loader=train_loaders[cid],
                target_class=target_class,
                device=device,
                epochs=10,
                lr=0.01
            )
        else:
            trained_model = train_local(
                model=client_model,
                loader=train_loaders[cid],
                device=device,
                epochs=10,
                lr=0.01
            )

        local_weights.append(trained_model.state_dict())

    # Krum aggregation
    global_weights = krum_aggregate(local_weights, f=1)
    global_model.load_state_dict(global_weights)

    # Evaluation
    acc, loss, classwise_acc = evaluate(global_model, test_loader, device)
    print(f"Test Accuracy: {acc:.4f} | Loss: {loss:.4f}")
    d["krum_overall"].append(acc)
    d["krum_target"].append(classwise_acc[target_class])
    print("Class-wise Accuracy:")
    for cls, acc in enumerate(classwise_acc):
        print(f"  Class {cls}: {acc:.4f}")


[Round 1]
Test Accuracy: 0.2771 | Loss: 6.6385
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.0000
  Class 2: 0.0000
  Class 3: 0.0000
  Class 4: 0.0000
  Class 5: 0.9890
  Class 6: 0.9980
  Class 7: 0.7840
  Class 8: 0.0000
  Class 9: 0.0000

[Round 2]
Test Accuracy: 0.2863 | Loss: 7.7417
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.0000
  Class 2: 0.0000
  Class 3: 0.0000
  Class 4: 0.0000
  Class 5: 0.9840
  Class 6: 0.9990
  Class 7: 0.8800
  Class 8: 0.0000
  Class 9: 0.0000

[Round 3]
Test Accuracy: 0.2893 | Loss: 7.2088
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.0000
  Class 2: 0.0000
  Class 3: 0.0000
  Class 4: 0.0000
  Class 5: 0.9870
  Class 6: 0.9990
  Class 7: 0.9070
  Class 8: 0.0000
  Class 9: 0.0000

[Round 4]
Test Accuracy: 0.2852 | Loss: 7.4784
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.0000
  Class 2: 0.0000
  Class 3: 0.0000
  Class 4: 0.0000
  Class 5: 0.9930
  Class 6: 0.9980
  Class 7: 0.8610
  Class 8: 0.0000
  Class 9: 0.0000

[Ro

In [35]:
d

{'baseline_overall': [0.3613,
  0.4576,
  0.536,
  0.5771,
  0.609,
  0.6443,
  0.6707,
  0.7023,
  0.7259,
  0.7451,
  0.7466,
  0.7659,
  0.7733,
  0.7809,
  0.7905],
 'baseline_target': [0.761,
  0.916,
  0.835,
  0.793,
  0.83,
  0.846,
  0.853,
  0.89,
  0.904,
  0.916,
  0.887,
  0.868,
  0.877,
  0.894,
  0.887],
 'attack_overall': [0.3063,
  0.4472,
  0.4841,
  0.5025,
  0.5436,
  0.5454,
  0.5951,
  0.6026,
  0.628,
  0.6373,
  0.6438,
  0.6495,
  0.6767,
  0.6779,
  0.688],
 'attack_target': [0.0,
  0.0,
  0.0,
  0.0,
  0.002,
  0.0,
  0.002,
  0.001,
  0.002,
  0.0,
  0.0,
  0.0,
  0.001,
  0.003,
  0.002],
 'def_overall': [0.3055,
  0.4416,
  0.4967,
  0.5041,
  0.5389,
  0.5335,
  0.5226,
  0.5212,
  0.5404,
  0.5591,
  0.5302,
  0.5643,
  0.5687,
  0.556,
  0.5783],
 'def_target': [0.0,
  0.0,
  0.001,
  0.001,
  0.0,
  0.448,
  0.364,
  0.309,
  0.359,
  0.393,
  0.302,
  0.42,
  0.467,
  0.372,
  0.589],
 'krum_overall': [0.2771,
  0.2863,
  0.2893,
  0.2852,
  0.2912,


In [7]:
def trimmed_mean_aggregate(weight_list, n_trim):
    n_clients = len(weight_list)

    all_keys = [set(w.keys()) for w in weight_list]
    common_keys = set.intersection(*all_keys)

    aggregated_weights = {}

    for key in common_keys:
        try:
            stacked = torch.stack([client[key] for client in weight_list], dim=0)  # shape: (n_clients, ...)
            sorted_vals, _ = torch.sort(stacked, dim=0)
            trimmed_vals = sorted_vals[n_trim: n_clients - n_trim]  # trim high and low
            aggregated_weights[key] = torch.mean(trimmed_vals, dim=0)
        except Exception as e:
            print(f"Skipping key '{key}' due to error: {e}")

    return aggregated_weights


t = {"overall":[], "target":[]}

global_model = FMNISTCNN()
device = "cuda" if torch.cuda.is_available() else "cpu"
global_model.to(device)

num_rounds = 15
num_clients = 5

# Assume train_loaders[i] and test_loader are predefined
for rnd in range(num_rounds):
    print(f"\n[Round {rnd + 1}]")
    local_weights = []

    for cid in range(num_clients):
        client_model = copy.deepcopy(global_model)

        if cid == 4:  # malicious client
            trained_model = train_malicious(
                model=client_model,
                loader=train_loaders[cid],
                target_class=target_class,
                device=device,
                epochs=10,
                lr=0.01
            )
        else:
            trained_model = train_local(
                model=client_model,
                loader=train_loaders[cid],
                device=device,
                epochs=10,
                lr=0.01
            )

        local_weights.append(trained_model.state_dict())

    global_weights = trimmed_mean_aggregate(local_weights, 2)
    global_model.load_state_dict(global_weights)

    # Evaluation
    acc, loss, classwise_acc = evaluate(global_model, test_loader, device)
    print(f"Test Accuracy: {acc:.4f} | Loss: {loss:.4f}")
    t["overall"].append(acc)
    t["target"].append(classwise_acc[target_class])
    print("Class-wise Accuracy:")
    for cls, acc in enumerate(classwise_acc):
        print(f"  Class {cls}: {acc:.4f}")


[Round 1]
Test Accuracy: 0.2925 | Loss: 2.0011
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.0000
  Class 2: 0.0000
  Class 3: 0.0000
  Class 4: 0.1570
  Class 5: 0.0010
  Class 6: 0.9850
  Class 7: 0.8600
  Class 8: 0.0000
  Class 9: 0.9220

[Round 2]
Test Accuracy: 0.3497 | Loss: 1.8038
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.0020
  Class 2: 0.0060
  Class 3: 0.0220
  Class 4: 0.8710
  Class 5: 0.0400
  Class 6: 0.6850
  Class 7: 0.9690
  Class 8: 0.0080
  Class 9: 0.8940

[Round 3]
Test Accuracy: 0.3628 | Loss: 1.6478
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.0300
  Class 2: 0.0550
  Class 3: 0.0300
  Class 4: 0.8650
  Class 5: 0.0090
  Class 6: 0.7260
  Class 7: 0.9770
  Class 8: 0.0380
  Class 9: 0.8980

[Round 4]
Test Accuracy: 0.3672 | Loss: 1.6691
Class-wise Accuracy:
  Class 0: 0.0010
  Class 1: 0.0910
  Class 2: 0.0150
  Class 3: 0.0680
  Class 4: 0.9450
  Class 5: 0.0050
  Class 6: 0.6080
  Class 7: 0.9720
  Class 8: 0.0460
  Class 9: 0.9210

[Ro

In [8]:
import torch
import torch.nn.functional as F
import copy
def distill_knowledge(global_model, local_models, proxy_loader, device, distill_epochs=3, temperature=4.0):
    print(f"→ Starting distillation with T = {temperature}")
    global_model.train()
    optimizer = torch.optim.SGD(global_model.parameters(), lr=0.01)

    for epoch in range(distill_epochs):
        print(f"  [Distill Epoch {epoch+1}/{distill_epochs}]")
        for images, _ in proxy_loader:
            images = images.to(device)
            ensemble_logits = torch.zeros((images.size(0), 10), device=device)

            with torch.no_grad():
                for model in local_models:
                    model.eval()
                    logits = model(images)
                    ensemble_logits += F.softmax(logits / temperature, dim=1)

            ensemble_logits /= len(local_models)
            output = global_model(images)
            student_log_probs = F.log_softmax(output / temperature, dim=1)
            loss = F.kl_div(student_log_probs, ensemble_logits, reduction="batchmean") * (temperature ** 2)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    return global_model


# Main FL loop with defense
global_model = FMNISTCNN()
device = "cuda" if torch.cuda.is_available() else "cpu"
global_model.to(device)

num_rounds = 15
num_clients = 5

for rnd in range(num_rounds):
    print(f"\n[Round {rnd + 1}]")
    local_weights = []

    for cid in range(num_clients):
        client_model = copy.deepcopy(global_model)

        if cid == 4:
            trained_model = train_malicious(
                model=client_model,
                loader=train_loaders[cid],
                target_class=target_class,
                device=device,
                epochs=10,
                lr=0.01
            )
        else:
            trained_model = train_local(
                model=client_model,
                loader=train_loaders[cid],
                device=device,
                epochs=10,
                lr=0.01
            )

        local_weights.append(trained_model.state_dict())

    # Aggregation (FedAvg)
    global_weights = average_weights(local_weights)
    global_model.load_state_dict(global_weights)

    
    if rnd >= 5:
        proxy_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)
        local_models = []
        for state in local_weights:
            local_model = FMNISTCNN().to(device)
            local_model.load_state_dict(state)
            local_models.append(local_model)

        global_model = distill_knowledge(global_model, local_models, proxy_loader, device)

    # Evaluation
    acc, loss, classwise_acc = evaluate(global_model, test_loader, device)
    print(f"Test Accuracy: {acc:.4f} | Loss: {loss:.4f}")
    
    d["def_overall"].append(acc)
    d["def_target"].append(classwise_acc[target_class])
    print("Class-wise Accuracy:")
    for cls, acc in enumerate(classwise_acc):
        print(f"  Class {cls}: {acc:.4f}")



[Round 1]
Test Accuracy: 0.3347 | Loss: 1.9384
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.0000
  Class 2: 0.0000
  Class 3: 0.0000
  Class 4: 0.7710
  Class 5: 0.0000
  Class 6: 0.8190
  Class 7: 0.8140
  Class 8: 0.0000
  Class 9: 0.9430

[Round 2]
Test Accuracy: 0.4090 | Loss: 1.5662
Class-wise Accuracy:
  Class 0: 0.0020
  Class 1: 0.6290
  Class 2: 0.0000
  Class 3: 0.0680
  Class 4: 0.9220
  Class 5: 0.0310
  Class 6: 0.6590
  Class 7: 0.7880
  Class 8: 0.0000
  Class 9: 0.9910

[Round 3]
Test Accuracy: 0.4767 | Loss: 1.3726
Class-wise Accuracy:
  Class 0: 0.0050
  Class 1: 0.9360
  Class 2: 0.0000
  Class 3: 0.1940
  Class 4: 0.9300
  Class 5: 0.0490
  Class 6: 0.6490
  Class 7: 0.8060
  Class 8: 0.2070
  Class 9: 0.9910

[Round 4]
→ Starting distillation with T = 4.0
  [Distill Epoch 1/3]
  [Distill Epoch 2/3]
  [Distill Epoch 3/3]
Test Accuracy: 0.5277 | Loss: 1.1700
Class-wise Accuracy:
  Class 0: 0.0030
  Class 1: 0.9290
  Class 2: 0.2460
  Class 3: 0.1570
  Class 4